## A Custom Convolutional Neural Network (CNN) Model to Classify BCN20000 Dataset

BCN20000 is a dataset consisting of dermoscopic images collated at the Hospital Clínic in Barcelona, Spain between 2010 and 2016. The dataset consists of eight key diagnostic categories in demoscopy such as melanoma and basal cell carcinoma. The dataset can be found [here🡵](https://figshare.com/articles/journal_contribution/BCN20000_Dermoscopic_Lesions_in_the_Wild/24140028/1) and the details are described in a peer reviewed article [here🡵](https://www.nature.com/articles/s41597-024-03387-w).

In this notebook, a custom deep lerning (CNN) model is constructed using PyTorch with the aim of classifying the images to the diagnostic category. The training data is augmented using random horizontal flip, random rotation and adjusting brightness. Data is also resized, cropped and normalized. 

**Dependencies:** 

The notebook assumes
* The training set images from BCN20000 are in a directory `train` within the root directory.
* The metadata file, `bcn_20k_train.csv` is in the root directory
* The BCN20000 test dataset is not labelled, so the train dataset is split for validation.

**Todo:**
* Run more epochs to see whether model performance improves further.
* Analyzing model predictions (confusion matrix etc.)
* Handling imbalanced data. Test weighted sampling or a different loss function.
* Make the code more robust by introducing error handling (e.g. while loading input images) and adding logs.
* Compare other neural network models (EfficientNet, [ResNet🡵](https://github.com/ngpraveen/Deep-Learning-models-for-BCN20000-dataset-classification/blob/main/ResNet18-Model_for_BCN20000_dataset.ipynb), ...)
  


In [1]:
import pandas as pd
import os

from PIL import Image

import torch
from torch import nn as nn
from torch import optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
from torchvision import transforms


In [2]:
root_dir = '/mnt/c/Users/prave/data/BCN20K_figshare/'

target_map = {'NV': 0, 'MEL': 1, 'BCC': 2, 'BKL': 3, 'AK': 4, 'SCC': 5, 'DF': 6, 'VASC': 7}

# decides whether to use weighted samples for imbalanced data
use_weighted_samples = False


### 1. Inspecting Metadata File

In [3]:
df_train = pd.read_csv(os.path.join(root_dir, "bcn_20k_train.csv"))
# df_test = pd.read_csv(os.path.join(root_dir, "bcn_20k_test.csv"))

In [4]:
df_train.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train


In [5]:
df_train['target'] = df_train['diagnosis'].map(target_map)

In [6]:
df_train.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split,target
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train,1
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train,1
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train,5
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train,0
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train,2


### 2. Create a Custom Dataset

In [8]:
class BCNDataset(Dataset):
    """
    Creates a custom dataset class that inherits from PyTorch's Dataset class.
    The metadata content (bcn_20k_train.csv) is added to an attribute `metadata`.
    The output class is defined in the column `diagnosis` which is coded to 0, 1, 2, ...
    as defined in `target_map`.

    Args:
        root_dir (str): The root directory where the dataset is stored. 
        transform (callable, optional): Optional transform to be applied to the input data.
    """
    
    def __init__(self, root_dir: str, transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, "train")
        self.transform = transform
        self.metadata = self.load_metadata()


    def __len__(self):
        return len(self.metadata)


    def __getitem__(self, idx:int):
        image, label = self.retrieve_image(idx)
        if self.transform:
            image = self.transform(image)        

        return image, label


    def load_metadata(self):
        metadata = pd.read_csv(
            os.path.join(self.root_dir, "bcn_20k_train.csv")
        )
        
        target_map = {'NV': 0, 'MEL': 1, 'BCC': 2, 'BKL': 3, 'AK': 4, 'SCC': 5, 'DF': 6, 'VASC': 7}
        metadata['target'] = metadata['diagnosis'].map(target_map)
        return metadata
    

    def retrieve_image(self, idx: int):
        image_name = self.metadata["bcn_filename"].iloc[idx]
        image_path = os.path.join(self.image_dir, image_name) 
        label = self.metadata["target"].iloc[idx]
        with Image.open(image_path) as img:
            image = img.convert("RGB")
        return image, label

In [9]:
dataset_row = BCNDataset(root_dir)

In [10]:
dataset_row.metadata

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split,target
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train,1
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train,1
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train,5
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train,0
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train,2
...,...,...,...,...,...,...,...,...,...
12408,BCN_0000020348.jpg,85.0,head/neck,BCC,BCN_0003925,2013-03-05,female,train,2
12409,BCN_0000020349.jpg,65.0,anterior torso,BKL,BCN_0001819,2016-05-05,male,train,3
12410,BCN_0000020350.jpg,70.0,lower extremity,MEL,BCN_0001085,2015-01-29,male,train,1
12411,BCN_0000020352.jpg,55.0,palms/soles,NV,BCN_0002083,2016-05-08,female,train,0


### Split Dataset and Create a Dataset Subset

In [11]:
def split_dataset(dataset, val_fraction=0.15, test_fraction=0.15):
    """
    Randomly split dataset into train, validation and test datasets.
    By default, splits in to 70%, 15% and 15%. 

    Args:
        dataset: The dataset object which is split into train, validation and test datasets.
        val_fraction (float, optional): Fraction of the dataset to be included in the validation set.
        test_fraction (float, optional): Fraction of the dataset to be included in the test set.  
    """
    
    total_size = len(dataset)
    val_size = int(total_size * val_fraction)
    test_size = int(total_size * test_fraction)
    train_size = total_size - val_size - test_size

    train_dataset, val_dataset, test_dataset = random_split(dataset, 
                                                            [train_size, val_size, test_size])
    return train_dataset, val_dataset, test_dataset

In [12]:
train_dataset1, val_dataset1, test_dataset1 = split_dataset(dataset_row)
len(train_dataset1), len(val_dataset1), len(test_dataset1)

(8691, 1861, 1861)

In [13]:
class SubsetWithTransform(Dataset):
    """
    Creates a subset class from BCNDataset objects. 
    Helps with applying different transforms to the train, validation 
    and test datasets. 
    """
    
    def __init__(self, subset, transform=None):
        print(subset)
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]
        if self.transform:
            image = self.transform(image)

        return image, label

    

In [14]:
# define different transforms for train and validation+test sets. 
# Train dataset is augmented with horizontal flip, random rotation etc. 
# But validation and test sets are not augmented. 
mean, std = [0.6125, 0.5277, 0.5061], [0.4241, 0.3242, 0.3054]
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2),
    transforms.Resize((256, 256)),  # Resize images to 256x256 pixels
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# for validation and test sets.
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to 256x256 pixels
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [15]:
train_dataset = SubsetWithTransform(train_dataset1, transform=train_transform)
val_dataset = SubsetWithTransform(val_dataset1, transform=val_transform)
test_dataset = SubsetWithTransform(test_dataset1, transform=val_transform)

### Create a Custom Neural Network

In [16]:
# define a custom CNN 
class BCN20DNN(nn.Module):
    """
    A convolutional neural network (CNN) model.

    The architecture consists of three convolutional blocks followed by 
    two fully connected layers. 

    Args:
        num_classes (int): Number of classes in the training dataset. 
    """
    def __init__(self, num_classes):
        super(BCN20DNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten4 = nn.Flatten()

        self.fc5a = nn.Linear(128 * 28 * 28, 512)
        self.relu5 = nn.ReLU()
        self.dropout5 = nn.Dropout(0.5)
        self.fc5b = nn.Linear(512, num_classes)

    def forward(self, x):
        # conv block 1
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # conv block 2
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # conv block 3
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.flatten4(x)

        # fully connected layers
        x = self.fc5a(x)
        x = self.relu5(x)
        x = self.dropout5(x)
        x = self.fc5b(x)

        return x

In [17]:
num_classes = dataset_row.metadata.target.nunique()

model1 = BCN20DNN(num_classes)

### Define Training Loop

The model uses cross entropy for loss function and Adam optimizer for updating parameter weights. 

In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [19]:
def training_loop(model, train_loader, val_loader, loss_function, optimizer, num_epochs, device):
    """
    Trains and validates neural network model. 

    Args:
        model: A neural network model object.
        train_loader: DataLoader for a training dataset.
        val_loader: DataLoader for the validation dataset.
        loss_function (callable): The loss function used during training.
        optimizer: The optimizer algorithm used to udpate parameter weights. 
        num_epochs (int): Number of epochs to train the model.
        device (torch.devie object): The device to run the training on. 
        
    """
    model.to(device)

    train_losses = []
    val_losses = []
    val_accuracies = []

    print("------------Training-------------")

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for idx, (images, labels) in enumerate(train_loader):
            print(idx, end="\r")
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)

        
        model.eval()
        running_val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss = loss_function(outputs, labels)
                running_val_loss += loss.item() * images.size(0)
                _, predicted_indices = torch.max(outputs, 1)
                total += labels.size(0)
                correct += predicted_indices.eq(labels).sum().item()

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)

        epoch_accuracy = 100.0 * correct / total
        val_accuracies.append(epoch_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_accuracy:.2f}%")
    print("--------Finished Training---------")

    metrics = [train_losses, val_losses, val_accuracies]
    
    # Return the trained model and the collected metrics
    return model, metrics

In [21]:
# For imbalanced data, if weighted sampling is used.
if use_weighted_samples:
    class_counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0,}
    for i in range(len(train_dataset)):
        d, c = train_dataset[i]
        class_counts[c] += 1
        if i%100 == 0:
            print(i, end="\r")

    class_weights = [1.0/class_counts[i] for i in range(8)]
    print(class_weights)

    sampler = WeightedRandomSampler(
        weights = class_weights,
        num_samples = len(train_dataset),
        replacement = True
    )

In [22]:
batch_size = 64

# if sampler is used for weighted sampling, turn shuffle off. 
if use_weighted_samples:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, shuffle=False)
else:
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [23]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

### Training

In [24]:
# training loop
model1, metrics = training_loop(
    model1, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_function=loss_function,
    optimizer=optimizer,
    num_epochs=10,
    device=adevice,
)


------------Training-------------
Epoch [1/10], Train Loss: 1.5343, Val Loss: 1.0038, Val Accuracy: 48.90%
Epoch [2/10], Train Loss: 1.3911, Val Loss: 1.2191, Val Accuracy: 51.37%
Epoch [3/10], Train Loss: 1.3422, Val Loss: 1.1807, Val Accuracy: 53.84%
Epoch [4/10], Train Loss: 1.3046, Val Loss: 1.5320, Val Accuracy: 53.41%
Epoch [5/10], Train Loss: 1.2752, Val Loss: 1.3389, Val Accuracy: 55.40%
Epoch [6/10], Train Loss: 1.2500, Val Loss: 1.2916, Val Accuracy: 56.37%
Epoch [7/10], Train Loss: 1.2304, Val Loss: 1.3767, Val Accuracy: 55.94%
Epoch [8/10], Train Loss: 1.1978, Val Loss: 1.2568, Val Accuracy: 56.80%
Epoch [9/10], Train Loss: 1.1786, Val Loss: 1.2032, Val Accuracy: 57.23%
Epoch [10/10], Train Loss: 1.1359, Val Loss: 0.9973, Val Accuracy: 57.71%
--------Finished Training---------


In [26]:
model1, metrics = training_loop(
    model1, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_function=loss_function,
    optimizer=optimizer,
    num_epochs=10,
    device=device,
)


------------Training-------------
Epoch [1/10], Train Loss: 1.1107, Val Loss: 1.0818, Val Accuracy: 56.90%
Epoch [2/10], Train Loss: 1.0772, Val Loss: 1.3663, Val Accuracy: 58.52%
Epoch [3/10], Train Loss: 1.0423, Val Loss: 1.3133, Val Accuracy: 57.87%
Epoch [4/10], Train Loss: 1.0064, Val Loss: 1.0384, Val Accuracy: 58.09%
Epoch [5/10], Train Loss: 0.9533, Val Loss: 0.8225, Val Accuracy: 58.09%
Epoch [6/10], Train Loss: 0.9237, Val Loss: 0.7291, Val Accuracy: 58.57%
Epoch [7/10], Train Loss: 0.8805, Val Loss: 0.8117, Val Accuracy: 58.73%
Epoch [8/10], Train Loss: 0.8177, Val Loss: 0.7525, Val Accuracy: 58.09%
Epoch [9/10], Train Loss: 0.7769, Val Loss: 0.6568, Val Accuracy: 58.89%
Epoch [10/10], Train Loss: 0.7422, Val Loss: 0.6323, Val Accuracy: 60.18%
--------Finished Training---------


In [27]:
model1, metrics = training_loop(
    model1, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_function=loss_function,
    optimizer=optimizer,
    num_epochs=10,
    device=device,
)

------------Training-------------
Epoch [1/10], Train Loss: 0.6887, Val Loss: 0.7093, Val Accuracy: 59.11%
Epoch [2/10], Train Loss: 0.6573, Val Loss: 0.7041, Val Accuracy: 60.40%
Epoch [3/10], Train Loss: 0.6170, Val Loss: 0.5584, Val Accuracy: 59.32%
Epoch [4/10], Train Loss: 0.5837, Val Loss: 0.3299, Val Accuracy: 59.32%
Epoch [5/10], Train Loss: 0.5413, Val Loss: 0.5957, Val Accuracy: 59.75%
Epoch [6/10], Train Loss: 0.4982, Val Loss: 0.5530, Val Accuracy: 59.91%
Epoch [7/10], Train Loss: 0.4854, Val Loss: 0.3427, Val Accuracy: 59.91%
Epoch [8/10], Train Loss: 0.4660, Val Loss: 0.4388, Val Accuracy: 60.83%
Epoch [9/10], Train Loss: 0.4454, Val Loss: 0.5023, Val Accuracy: 60.51%
Epoch [10/10], Train Loss: 0.3971, Val Loss: 0.3617, Val Accuracy: 62.44%
--------Finished Training---------


In [28]:
model1, metrics = training_loop(
    model1, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    loss_function=loss_function,
    optimizer=optimizer,
    num_epochs=10,
    device=device,
)

------------Training-------------
Epoch [1/10], Train Loss: 0.3846, Val Loss: 0.3615, Val Accuracy: 61.04%
Epoch [2/10], Train Loss: 0.3699, Val Loss: 0.4991, Val Accuracy: 60.83%
Epoch [3/10], Train Loss: 0.3286, Val Loss: 0.4042, Val Accuracy: 62.01%
Epoch [4/10], Train Loss: 0.3271, Val Loss: 0.3168, Val Accuracy: 61.74%
Epoch [5/10], Train Loss: 0.3001, Val Loss: 0.2240, Val Accuracy: 61.26%
Epoch [6/10], Train Loss: 0.3180, Val Loss: 0.4316, Val Accuracy: 61.63%
Epoch [7/10], Train Loss: 0.2966, Val Loss: 0.2395, Val Accuracy: 61.63%
Epoch [8/10], Train Loss: 0.2786, Val Loss: 0.4047, Val Accuracy: 61.58%
Epoch [9/10], Train Loss: 0.2673, Val Loss: 0.0972, Val Accuracy: 61.20%
Epoch [10/10], Train Loss: 0.2541, Val Loss: 0.1945, Val Accuracy: 60.24%
--------Finished Training---------
